# Import Required Libraries
Import the necessary libraries, including pandas, numpy, and matplotlib.

In [23]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the Dataset
Load the electric vehicle dataset from the CSV file using pandas.

In [29]:
# Load the Dataset
df = pd.read_csv('Electric_Vehicle_Population_Data.csv')

# Display the first few rows of the dataframe to understand its structure
df.head(20)

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,KM8K33AGXL,King,Seattle,WA,98103.0,2020,HYUNDAI,KONA,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,258,0,43.0,249675142,POINT (-122.34301 47.659185),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
1,1C4RJYB61N,King,Bothell,WA,98011.0,2022,JEEP,GRAND CHEROKEE,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25,0,1.0,233928502,POINT (-122.20578 47.762405),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
2,1C4RJYD61P,Yakima,Yakima,WA,98908.0,2023,JEEP,GRAND CHEROKEE,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25,0,14.0,229675939,POINT (-120.6027202 46.5965625),PACIFICORP,5.307700e+10
3,5YJ3E1EA7J,King,Kirkland,WA,98034.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,45.0,104714466,POINT (-122.209285 47.71124),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
4,WBY7Z8C5XJ,Thurston,Olympia,WA,98501.0,2018,BMW,I3,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,97,0,22.0,185498386,POINT (-122.89692 47.043535),PUGET SOUND ENERGY INC,5.306701e+10
5,5YJ3E1EAXL,Snohomish,Marysville,WA,98271.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,266,0,38.0,124595523,POINT (-122.1713847 48.10433),PUGET SOUND ENERGY INC,5.306194e+10
6,2C4RC1N77H,King,Kent,WA,98042.0,2017,CHRYSLER,PACIFICA,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,33,0,47.0,1815593,POINT (-122.111625 47.36078),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
7,5YJYGDEE3L,King,Woodinville,WA,98072.0,2020,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,291,0,45.0,124760555,POINT (-122.151665 47.75855),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10
8,5YJ3E1EA1J,Island,Coupeville,WA,98239.0,2018,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,215,0,10.0,125048003,POINT (-122.6880708 48.2179983),PUGET SOUND ENERGY INC,5.302997e+10
9,7SAYGDEF0P,King,Bellevue,WA,98004.0,2023,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,48.0,240416207,POINT (-122.201905 47.61385),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303302e+10


In [30]:
# Assuming the column for range is named 'Range', find the maximum value
max_range = df['Electric Range'].max()

print(f"The maximum range value is: {max_range}")

The maximum range value is: 337


# Explore the Dataset
Perform initial exploration of the dataset, including checking for missing values and basic statistics.

In [25]:
# Explore the Dataset

# Check for missing values in the dataset
missing_values = df.isnull().sum()
print("Missing values in each column:\n", missing_values)

# Display basic statistics of the dataset
basic_stats = df.describe()
print("\nBasic statistics of the dataset:\n", basic_stats)

# Display the data types of each column
data_types = df.dtypes
print("\nData types of each column:\n", data_types)

# Display the shape of the dataset
dataset_shape = df.shape
print("\nShape of the dataset:", dataset_shape)

Missing values in each column:
 VIN (1-10)                                             0
County                                                 3
City                                                   3
State                                                  0
Postal Code                                            3
Model Year                                             0
Make                                                   0
Model                                                  0
Electric Vehicle Type                                  0
Clean Alternative Fuel Vehicle (CAFV) Eligibility      0
Electric Range                                         0
Base MSRP                                              0
Legislative District                                 341
DOL Vehicle ID                                         0
Vehicle Location                                       7
Electric Utility                                       3
2020 Census Tract                                      3

# Data Cleaning
Clean the dataset by handling missing values, correcting data types, and removing duplicates.

In [26]:
# Data Cleaning

# Handle missing values by filling with appropriate values or dropping rows/columns
df.fillna({
    'column_name_1': 'default_value',  # Replace 'column_name_1' with actual column name and 'default_value' with appropriate value
    'column_name_2': df['column_name_2'].mean()  # Replace 'column_name_2' with actual column name and use mean for numerical columns
}, inplace=True)

# Drop rows with any remaining missing values
df.dropna(inplace=True)

# Correct data types if necessary
df['column_name_3'] = df['column_name_3'].astype('int')  # Replace 'column_name_3' with actual column name and 'int' with appropriate data type

# Remove duplicates
df.drop_duplicates(inplace=True)

# Verify the cleaning process
print("\nMissing values after cleaning:\n", df.isnull().sum())
print("\nData types after cleaning:\n", df.dtypes)
print("\nShape of the dataset after cleaning:", df.shape)

KeyError: 'column_name_2'

# Data Visualization
Create visualizations to understand the distribution and relationships in the data using matplotlib and seaborn.

In [ ]:
# Data Visualization

import seaborn as sns

# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Plot the distribution of a numerical column (e.g., 'Battery Capacity')
plt.figure(figsize=(10, 6))
sns.histplot(df['Battery Capacity'], kde=True, bins=30)
plt.title('Distribution of Battery Capacity')
plt.xlabel('Battery Capacity (kWh)')
plt.ylabel('Frequency')
plt.show()

# Plot the relationship between two numerical columns (e.g., 'Range' and 'Battery Capacity')
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Battery Capacity', y='Range', data=df)
plt.title('Range vs Battery Capacity')
plt.xlabel('Battery Capacity (kWh)')
plt.ylabel('Range (km)')
plt.show()

# Plot the count of a categorical column (e.g., 'Brand')
plt.figure(figsize=(12, 8))
sns.countplot(y='Brand', data=df, order=df['Brand'].value_counts().index)
plt.title('Count of Electric Vehicles by Brand')
plt.xlabel('Count')
plt.ylabel('Brand')
plt.show()

# Plot the boxplot to show the distribution of 'Range' across different 'Brand'
plt.figure(figsize=(14, 8))
sns.boxplot(x='Range', y='Brand', data=df)
plt.title('Range Distribution by Brand')
plt.xlabel('Range (km)')
plt.ylabel('Brand')
plt.show()

# Statistical Analysis
Perform statistical analysis to derive insights from the dataset.

In [ ]:
# Statistical Analysis

# Calculate the mean, median, and standard deviation of the 'Range' column
mean_range = df['Range'].mean()
median_range = df['Range'].median()
std_range = df['Range'].std()

print(f"Mean Range: {mean_range}")
print(f"Median Range: {median_range}")
print(f"Standard Deviation of Range: {std_range}")

# Calculate the correlation matrix to understand relationships between numerical columns
correlation_matrix = df.corr()
print("\nCorrelation Matrix:\n", correlation_matrix)

# Identify the top 5 brands with the highest average range
top_brands_by_range = df.groupby('Brand')['Range'].mean().sort_values(ascending=False).head(5)
print("\nTop 5 Brands by Average Range:\n", top_brands_by_range)

# Perform a t-test to compare the range of two specific brands (e.g., 'Brand A' and 'Brand B')
from scipy.stats import ttest_ind

brand_a_range = df[df['Brand'] == 'Brand A']['Range']
brand_b_range = df[df['Brand'] == 'Brand B']['Range']

t_stat, p_value = ttest_ind(brand_a_range, brand_b_range, nan_policy='omit')
print(f"\nT-test between Brand A and Brand B Range:\nT-statistic: {t_stat}, P-value: {p_value}")

# Calculate the percentage of electric vehicles with a range greater than 300 km
percentage_above_300km = (df[df['Range'] > 300].shape[0] / df.shape[0]) * 100
print(f"\nPercentage of Electric Vehicles with Range > 300 km: {percentage_above_300km:.2f}%")